In [ ]:
import numpy as np

# Bitplane from residual

In [ ]:
bitstring_data = [
    '00101001',
    '11010101',
    '00000101',
    '11101010',
    '10100101'
]

In [ ]:
data = np.array([int(x,2) for x in bitstring_data], dtype='uint8')
data

In [ ]:
[np.binary_repr(x) for x in data]

## With string operations

In [ ]:
minimal_string_repr = [np.binary_repr(x) for x in data]
minimal_string_repr_len = [len(x) for x in minimal_string_repr]
result = ''
while any([x != '' for x in minimal_string_repr]):
    for i in range(len(minimal_string_repr)):
        if minimal_string_repr[i] != "":
            result += minimal_string_repr[i][0]
            minimal_string_repr[i] = minimal_string_repr[i][1:]
result = [result[i:i+8] for i in range(0,len(result),8)]
result[-1] = "0"*(8 - len(result[-1])) + result[-1]
result = [int(x,2) for x in result]

In [ ]:
minimal_string_repr_len, result

In [ ]:
# split = [1] * len(minimal_string_repr_len)
# result = result[len(minimal_string_repr_len) - 1:]
# minimal_string_repr_len = [x - 1 for x in minimal_string_repr_len]

result = "".join([np.binary_repr(x) for x in result])
number_of_values = len(minimal_string_repr_len)
split = [0] * number_of_values
while any([x > 0 for x in minimal_string_repr_len]):
    for i in range(number_of_values):
        if minimal_string_repr_len[i] > 0:
            bit = result[0]
            result = result[1:]
            split[i] <<= 1
            split[i] += int(bit)
            minimal_string_repr_len[i] -= 1

In [ ]:
split

## with numbers

In [ ]:
data

In [ ]:
def to_bitplanes_unregular(data):
    block_sizes = [len(np.binary_repr(x)) for x in data]
    result = []
    ix = 0
    val = 0
    while any([x>0 for x in block_sizes]):
        block_sizes = [x-1 for x in block_sizes]
        for i in range(len(block_sizes)):
            if block_sizes[i] >= 0:
                bit = (data[i] & (1 << block_sizes[i])) > 0
                val <<= 1
                if bit: 
                    val += 1
                ix += 1
                if ix == 8:
                    result.append(val)
                    val = 0
                    ix = 0
    push = 8 - len(np.binary_repr(val))
    result.append(val << push)
    return [len(np.binary_repr(x)) for x in data], result, push

In [ ]:
def from_bitplanes_unregular(sizes, bits, push):
    bitstream = [bits[s] & (1 << (8 - powr - 1)) > 0 for s in range(len(bits)) for powr in range(8)][:-push]
    split = [0] * len(sizes)
    while any([x > 0 for x in sizes]):
        for i in range(len(sizes)):
            if sizes[i] > 0:
                bit = bitstream[0]
                bitstream = bitstream[1:]
                split[i] <<= 1
                split[i] += int(bit)
                sizes[i] -= 1
    return np.array(split)

In [ ]:
from_bitplanes_unregular(*to_bitplanes_unregular(data))

# Bitplane from u8

In [ ]:
def to_bitplanes_for_u8(data):
    result = []
    scanned = 0
    pos = 7
    value = 0
    while pos >= 0:
        check_value = 1 << pos
        for i in range(data.size):
            value <<= 1
            value += int((check_value & data[i]) > 0)
            scanned += 1
            if scanned == 8:
                result.append(value)
                value = 0
                scanned = 0
        pos -= 1
    return np.array(result, dtype='uint8')

In [ ]:
def from_bitplanes_for_u8(data):
    split = [0] * data.size
    for i in range(5*8):
        elem = i%5
        powr = 8 - i//5 - 1
        elem_orig = i//8
        powr_orig = 8 - i%8 - 1
        if (data[elem_orig] & (1 << powr_orig)) > 0:
            split[elem] += (1 << powr)
    return np.array(split, dtype='uint8')

In [ ]:
data

In [ ]:
to_bitplanes_for_u8(data)

In [ ]:
from_bitplanes_for_u8(to_bitplanes_for_u8(data))